# Data Processing


In [14]:
# %pip install datasets
# %pip install psycopg2
import pandas as pd


import psycopg2
import json


In [ ]:
small_df = pd.read_csv("hf://datasets/Sachinkelenjaguri/Resume_dataset/UpdatedResumeDataSet.csv")


In [12]:
roles = [
    'Data Science', 'Web Designing', 
    'Java Developer', 'Automation Testing',
    'Operations Manager', 'Python Developer',
    'DevOps Engineer', 'Network Security Engineer',
    'Database', 'DotNet Developer',
    'Testing', 'Blockchain'
]
small_df = small_df[small_df['Category'].isin(roles)]
small_df


,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \nMay 2013 to May 2017 B.E ...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \n MCA YMCAUST, Faridabad..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [ ]:
-- Ensure the table exists before vectorizing
DROP TABLE IF EXISTS resumes_1;

CREATE TABLE resumes_1 (
    id SERIAL PRIMARY KEY,
    category TEXT NOT NULL,
    content TEXT NOT NULL,
    metadata JSONB
);

-- Create the vectorizer using AI functions

SELECT ai.create_vectorizer(   
    'resumes_1'::regclass,
    formatting => ai.formatting_python_template('Job Role: $category\n\nCV:\n$chunk'),
    embedding => ai.embedding_ollama('deepseek-r1:7b', 3584),
    chunking => ai.chunking_recursive_character_text_splitter('content'),
    destination => 'resume_1_embedding'
);

In [15]:
DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": "5432" 
}


# Function to insert data
def insert_data(df):
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(**DB_PARAMS)
        cur = conn.cursor()
        
        # Insert each row into the database
        for _, row in df.iterrows():
            category = row['Category']
            content = row['Resume']
            metadata = json.dumps({"source": "uploaded_resume"})  # Example metadata

            cur.execute("""
                INSERT INTO resumes_1 (category, content, metadata)
                VALUES (%s, %s, %s)
            """, (category, content, metadata))

        # Commit changes
        conn.commit()
        print("Data inserted successfully.")

    except Exception as e:
        print("Error:", e)
    finally:
        cur.close()
        conn.close()

# Call the function to insert data
insert_data(small_df)


Data inserted successfully.


In [ ]:
big_df = pd.read_csv("hf://datasets/opensporks/resumes/Resume/Resume.csv")


In [ ]:
massive_df = pd.read_csv("hf://datasets/InferencePrince555/Resume-Dataset/updated_data_final_cleaned.csv")


In [1]:
CREATE OR REPLACE FUNCTION generate_rag_response(query_text TEXT)
RETURNS TEXT AS $$
DECLARE
   context_chunks TEXT;
   response TEXT;
BEGIN
   -- Perform similarity search to find relevant resumes
   SELECT string_agg(category || ': ' || chunk, E'\n') INTO context_chunks
   FROM
   (
       SELECT category, chunk
       FROM resume_1_embedding
       ORDER BY embedding <=> ai.ollama_embed('deepseek-r1:7b', query_text)
       LIMIT 3
   ) AS relevant_posts;

   -- Generate a summary using llama3
   SELECT ai.ollama_chat_complete
   ( 'deepseek-r1:7b'
   , jsonb_build_array
     ( jsonb_build_object('role', 'system', 'content', 'you are a helpful assistant')
     , jsonb_build_object
       ('role', 'user'
       , 'content', query_text || E'\nUse the following context to respond.\n' || context_chunks
       )
     )
   )->'message'->>'content' INTO response;

   RETURN response;
END;
$$ LANGUAGE plpgsql;

SyntaxError: invalid syntax (1555296760.py, line 1)